# MultiLevelSplittingStrategy 단독 테스트

이 노트북은 MultiLevelSplittingStrategy의 동작을 단계적으로 확인하고 테스트합니다.

## 목표
- PNS 대제목과 하위 섹션 purchaseState 연결 문제 해결 확인
- 다중 레벨 분할 전략의 효과 검증
- 계층적 컨텍스트 보존 확인


## 1. 환경 설정 및 라이브러리 임포트


In [99]:
import os
import sys
import re
from typing import List, Dict, Any, Tuple
from pathlib import Path

# 프로젝트 루트 경로 추가
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

print("✅ 라이브러리 임포트 완료")


✅ 라이브러리 임포트 완료


## 2. MultiLevelSplittingStrategy 클래스 정의


In [105]:
class MultiLevelSplittingStrategy:
    """다중 레벨 분할 전략 - 큰 단위 + 작은 단위 문서 생성"""
    
    def __init__(self, document_path: str):
        self.document_path = document_path
        self.raw_text = self._load_document()
        
    def _load_document(self) -> str:
        with open(self.document_path, 'r', encoding='utf-8') as f:
            return f.read()
    
    def split_documents(self) -> List[Document]:
        """다중 레벨로 문서 분할"""
        documents = []
        
        # 헤더 기반 분할
        header_splitter = MarkdownHeaderTextSplitter(
            headers_to_split_on=[
                ("#", "Header 1"),
                ("##", "Header 2"),
                ("###", "Header 3"),
                ("####", "Header 4")
            ]
        )
        header_docs = header_splitter.split_text(self.raw_text)
        
        # 계층별로 문서 그룹핑
        hierarchy_groups = self._group_by_hierarchy(header_docs)
        
        # 각 레벨별 문서 생성
        for level, group_docs in hierarchy_groups.items():
            level_docs = self._create_level_documents(group_docs, level)
            documents.extend(level_docs)
            print(f"📊 {level} 레벨: {len(level_docs)}개 문서 생성")
        
        return documents
    
    def _group_by_hierarchy(self, header_docs: List[Document]) -> Dict[str, List[Document]]:
        """계층별로 문서 그룹핑 - 개선된 로직"""
        groups: Dict[str, List[Document]] = {
            "major": [],     # 대제목 레벨 (H1)
            "medium": [],    # 중제목 레벨 (H2)
            "minor": [],     # 소제목 레벨 (H3, H4)
        }
        
        print(f"🔍 헤더 문서 분석 중... (총 {len(header_docs)}개)")
        
        # 문서별 상세 분석
        for i, doc in enumerate(header_docs):
            metadata = doc.metadata
            content_preview = doc.page_content[:100].replace('\n', ' ')
            
            # 첫 10개 문서의 상세 정보 출력
            if i < 10:
                print(f"\n📄 문서 #{i+1}:")
                print(f"  메타데이터: {metadata}")
                print(f"  내용: {content_preview}...")
            
            # 헤더 레벨 분류 (가장 구체적인 레벨 기준)
            has_h1 = "Header 1" in metadata and metadata.get("Header 1", "").strip()
            has_h2 = "Header 2" in metadata and metadata.get("Header 2", "").strip()
            has_h3 = "Header 3" in metadata and metadata.get("Header 3", "").strip()
            has_h4 = "Header 4" in metadata and metadata.get("Header 4", "").strip()
            
            # 가장 구체적인 헤더 레벨로 분류 (H4 > H3 > H2 > H1 순)
            if has_h4:
                groups["minor"].append(doc)
            elif has_h3:
                groups["minor"].append(doc)
            elif has_h2:
                groups["medium"].append(doc)
            elif has_h1:
                groups["major"].append(doc)
            else:
                # 메타데이터가 없는 문서들을 컨텐츠 기반으로 분류
                content_lines = doc.page_content.split('\n')
                classified = False
                
                for line in content_lines[:5]:  # 처음 5줄 확인
                    line = line.strip()
                    if line.startswith('# ') and not line.startswith('## '):
                        groups["major"].append(doc)
                        classified = True
                        break
                    elif line.startswith('## ') and not line.startswith('### '):
                        groups["medium"].append(doc)
                        classified = True
                        break
                    elif line.startswith('### ') or line.startswith('#### '):
                        groups["minor"].append(doc)
                        classified = True
                        break
                
                if not classified:
                    print(f"⚠️  분류 실패 #{i+1}: {content_preview}")
                    groups["minor"].append(doc)  # 기본적으로 minor에 할당
        
        print(f"\n🔍 최종 계층별 그룹핑 결과:")
        print(f"  - Major (H1): {len(groups['major'])}개")
        print(f"  - Medium (H2): {len(groups['medium'])}개")
        print(f"  - Minor (H3+): {len(groups['minor'])}개")
        
        return groups
    
    def _create_level_documents(self, docs: List[Document], level: str) -> List[Document]:
        """레벨별 문서 생성"""
        level_documents = []
        
        # 레벨별 청크 크기 설정
        chunk_sizes = {
            "major": 2000,    # 대제목: 큰 컨텍스트 보존
            "medium": 1200,   # 중제목: 중간 크기
            "minor": 800      # 소제목: 세밀한 분할
        }
        
        chunk_size = chunk_sizes.get(level, 1000)
        print(f"📏 {level} 레벨 청크 크기: {chunk_size}자")
        
        for doc in docs:
            # 계층적 제목 생성
            title_hierarchy = self._build_title_hierarchy(doc.metadata)
            
            # 컨텍스트 강화
            enhanced_content = self._enhance_with_context(doc.page_content, title_hierarchy, level)
            
            # 청킹
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=200,
                separators=["\\n\\n", "\\n", ". ", "? ", "! ", ", "]
            )
            chunks = text_splitter.split_text(enhanced_content)
            
            for i, chunk in enumerate(chunks):
                metadata = doc.metadata.copy()
                metadata.update({
                    "chunk_index": i,
                    "hierarchy_level": level,
                    "title_hierarchy": title_hierarchy,
                    "source_strategy": f"multi_level_{level}",
                    "chunk_size": len(chunk),
                    "contains_pns": self._check_pns_context(chunk, title_hierarchy)
                })
                level_documents.append(Document(page_content=chunk, metadata=metadata))
        
        return level_documents
    
    def _build_title_hierarchy(self, metadata: Dict) -> str:
        """제목 계층 구조 생성"""
        hierarchy_parts = []
        for level in range(1, 5):
            header_key = f"Header {level}"
            if header_key in metadata and metadata[header_key]:
                hierarchy_parts.append(metadata[header_key].strip())
        return " > ".join(hierarchy_parts) if hierarchy_parts else "Unknown"
    
    def _enhance_with_context(self, content: str, title_hierarchy: str, level: str) -> str:
        """레벨별 컨텍스트 강화"""
        # PNS 관련 섹션인지 확인
        is_pns_section = "PNS" in title_hierarchy.upper() or "PAYMENT NOTIFICATION" in title_hierarchy.upper()
        
        context_info = f"[계층]: {title_hierarchy}\\n"
        
        if is_pns_section:
            context_info += "[PNS 관련]: 이 내용은 PNS(Payment Notification Service) 결제알림서비스와 관련됩니다.\\n"
        
        context_info += f"[레벨]: {level}\\n\\n"
        
        return context_info + content
    
    def _check_pns_context(self, content: str, title_hierarchy: str) -> bool:
        """PNS 컨텍스트 여부 확인"""
        content_upper = content.upper()
        hierarchy_upper = title_hierarchy.upper()
        
        return ("PNS" in hierarchy_upper or 
                "PAYMENT NOTIFICATION" in hierarchy_upper or
                "PNS" in content_upper or
                "PAYMENT NOTIFICATION" in content_upper)

print("✅ MultiLevelSplittingStrategy 클래스 정의 완료")


✅ MultiLevelSplittingStrategy 클래스 정의 완료


## 3. 문서 로드 및 기본 정보 확인


In [106]:
# 문서 경로 설정
document_path = "data/dev_center_guide_allmd_touched.md"

# 문서 존재 확인
if not os.path.exists(document_path):
    print(f"❌ 문서를 찾을 수 없습니다: {document_path}")
else:
    print(f"✅ 문서 발견: {document_path}")
    
    # 문서 크기 확인
    file_size = os.path.getsize(document_path)
    print(f"📄 파일 크기: {file_size:,} bytes ({file_size/1024/1024:.2f} MB)")
    
    # 문서 줄 수 확인
    with open(document_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        print(f"📝 총 줄 수: {len(lines):,}줄")
        
    # PNS 관련 섹션 미리보기
    with open(document_path, 'r', encoding='utf-8') as f:
        content = f.read()
        pns_matches = len(re.findall(r'PNS|Payment Notification', content, re.IGNORECASE))
        purchase_matches = len(re.findall(r'purchaseState', content, re.IGNORECASE))
        print(f"🔍 PNS 관련 키워드: {pns_matches}개")
        print(f"🔍 purchaseState 키워드: {purchase_matches}개")


✅ 문서 발견: data/dev_center_guide_allmd_touched.md
📄 파일 크기: 470,535 bytes (0.45 MB)
📝 총 줄 수: 8,933줄
🔍 PNS 관련 키워드: 29개
🔍 purchaseState 키워드: 43개


## 3.5. 문서 구조 디버깅 (문제 해결용)


In [107]:
# 문서 구조 정밀 분석
print("🔍 원본 문서 헤더 구조 분석")
print("=" * 60)

# 원본 문서에서 실제 헤더 패턴 확인
with open(document_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

header_patterns = {"# ": 0, "## ": 0, "### ": 0, "#### ": 0}
sample_headers = {"# ": [], "## ": [], "### ": [], "#### ": []}

for i, line in enumerate(lines):
    line_stripped = line.strip()
    for pattern in header_patterns.keys():
        if line_stripped.startswith(pattern) and not line_stripped.startswith(pattern + "#"):
            header_patterns[pattern] += 1
            if len(sample_headers[pattern]) < 5:  # 각 레벨당 5개 샘플 수집
                sample_headers[pattern].append(line_stripped)

print("📊 실제 마크다운 헤더 통계:")
for pattern, count in header_patterns.items():
    level_name = {"# ": "H1", "## ": "H2", "### ": "H3", "#### ": "H4"}[pattern]
    print(f"  {level_name}: {count}개")

print("\n📋 헤더 샘플:")
for pattern, samples in sample_headers.items():
    level_name = {"# ": "H1", "## ": "H2", "### ": "H3", "#### ": "H4"}[pattern]
    if samples:
        print(f"\n{level_name} 샘플:")
        for sample in samples:
            print(f"  {sample}")

# MarkdownHeaderTextSplitter로 분할해서 실제 메타데이터 확인
from langchain_text_splitters import MarkdownHeaderTextSplitter

header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header 1"),
        ("##", "Header 2"), 
        ("###", "Header 3"),
        ("####", "Header 4")
    ]
)

with open(document_path, 'r', encoding='utf-8') as f:
    raw_content = f.read()

header_docs = header_splitter.split_text(raw_content)

print(f"\n🔍 MarkdownHeaderTextSplitter 분할 결과: {len(header_docs)}개 섹션")

# 메타데이터 분석
metadata_stats = {"has_h1": 0, "has_h2": 0, "has_h3": 0, "has_h4": 0, "empty": 0}

print("\n📄 처음 10개 섹션의 메타데이터:")
for i, doc in enumerate(header_docs[:10]):
    metadata = doc.metadata
    print(f"\n섹션 #{i+1}:")
    print(f"  메타데이터: {metadata}")
    
    # 메타데이터 통계
    if not metadata:
        metadata_stats["empty"] += 1
    else:
        if metadata.get("Header 1"):
            metadata_stats["has_h1"] += 1
        if metadata.get("Header 2"):
            metadata_stats["has_h2"] += 1
        if metadata.get("Header 3"):
            metadata_stats["has_h3"] += 1
        if metadata.get("Header 4"):
            metadata_stats["has_h4"] += 1
    
    # 내용 미리보기
    content_preview = doc.page_content[:150].replace('\n', ' ')
    print(f"  내용: {content_preview}...")

print(f"\n📊 메타데이터 통계 (처음 10개):")
for key, count in metadata_stats.items():
    print(f"  {key}: {count}개")


🔍 원본 문서 헤더 구조 분석
📊 실제 마크다운 헤더 통계:
  H1: 61개
  H2: 98개
  H3: 287개
  H4: 96개

📋 헤더 샘플:

H1 샘플:
  # 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드
  # 01. 원스토어 인앱결제 개요
  # 02. 사전준비
  # 03. 결제 테스트 및 보안
  # 04. 원스토어 인앱결제 SDK를 사용해 구현하기

H2 샘플:
  ## **원스토어 인앱결제란?** <a href="#id-01." id="id-01."></a>
  ## **인앱 상품의 유형** <a href="#id-01." id="id-01."></a>
  ## **결제 프로세스** <a href="#id-01." id="id-01."></a>
  ## **권장 개발 환경** <a href="#id-01." id="id-01."></a>
  ## **상품 등록하기**  <a href="#id-02." id="id-02."></a>

H3 샘플:
  ### 원스토어 로그인 하기 <a href="#id-01." id="id-01."></a>
  ### **인앱 상품 구매 요청하기**  <a href="#id-01." id="id-01."></a>
  ### **구매 확인하기**  <a href="#id-01." id="id-01."></a>
  ### 구독형 상품 관리 <a href="#id-01." id="id-01."></a>
  ### **테스트 ID 등록하기**   <a href="#id-03.-id" id="id-03.-id"></a>

H4 샘플:
  #### **GaaSignInClient 초기화**
  #### **포그라운드 로그인하기**
  #### **백그라운드 로그인하기**
  #### v21.02.00 업데이트 – 글로벌 스토어 선택 기능 추가
  #### 사용자가 정기 결제를 업그레이드, 다운그레이드 또는 변경할 수 있도록 허용 <a href="#id-04.-sdk" id="id-04.-sdk

## 4. MultiLevelSplittingStrategy 실행 및 문서 분할


In [108]:
# MultiLevelSplittingStrategy 인스턴스 생성
print("🚀 MultiLevelSplittingStrategy 시작")
splitter = MultiLevelSplittingStrategy(document_path)

# 문서 분할 실행
print("\n📄 문서 분할 시작...")
documents = splitter.split_documents()

print(f"\n✅ 문서 분할 완료!")
print(f"📊 총 생성된 문서 수: {len(documents)}개")


🚀 MultiLevelSplittingStrategy 시작

📄 문서 분할 시작...
🔍 헤더 문서 분석 중... (총 495개)

📄 문서 #1:
  메타데이터: {}
  내용: 출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md...
⚠️  분류 실패 #1: 출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md

📄 문서 #2:
  메타데이터: {'Header 1': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드'}
  내용: 원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.   보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.   {% hint style="info"...

📄 문서 #3:
  메타데이터: {'Header 1': '01. 원스토어 인앱결제 개요', 'Header 2': '**원스토어 인앱결제란?** <a href="#id-01." id="id-01."></a>'}
  내용: 원스토어 인앱결제(In-App Purchase, IAP)는 안드로이드 앱 내에 구현된 상품을 원스토어의 인증 및 결제 시스템을 이용하여 사용자에게 판매, 청구하여 개발자에게 정산하...

📄 문서 #4:
  메타데이터: {'Header 1': '01. 원스토어 인앱결제 개요', 'Header 2': '**인앱 상품의 유형** <a href="#id-01." id="id-01."></a>'}
  내용: 원스토어 인앱결제 API V7(SDK V21)는 관리형 상품과 구독형 상품. 두 가지 유형의 인앱 상품을 제공 합니다.\ 우선, 각 타입이 어떤 특성을 가지고 있는지 확인하신 후 ...

📄 문서 #5:
  메타데이터: {'Header 1': '01. 원스토어 인앱결제 개요', 'Header 2': '**결제 프로세스** <a href="#id-01." id="id-01."></a>'}
  내용: 원스

## 5. 분할된 문서 분석


In [109]:
# 레벨별 문서 수 분석
level_counts = {}
pns_counts = {}
chunk_sizes = {}

for doc in documents:
    level = doc.metadata.get('hierarchy_level', 'unknown')
    
    # 레벨별 카운트
    level_counts[level] = level_counts.get(level, 0) + 1
    
    # PNS 관련 문서 카운트
    if doc.metadata.get('contains_pns', False):
        pns_counts[level] = pns_counts.get(level, 0) + 1
    
    # 청크 크기 분석
    size = doc.metadata.get('chunk_size', 0)
    if level not in chunk_sizes:
        chunk_sizes[level] = []
    chunk_sizes[level].append(size)

print("📊 레벨별 문서 분석 결과:")
print("=" * 50)

for level in ['major', 'medium', 'minor']:
    count = level_counts.get(level, 0)
    pns_count = pns_counts.get(level, 0)
    sizes = chunk_sizes.get(level, [])
    avg_size = sum(sizes) / len(sizes) if sizes else 0
    
    print(f"\n🔹 {level.upper()} 레벨:")
    print(f"  총 문서: {count}개")
    pns_percentage = pns_count/count*100 if count > 0 else 0.0
    print(f"  PNS 관련: {pns_count}개 ({pns_percentage:.1f}%)")
    print(f"  평균 크기: {avg_size:.0f}자")
    if sizes:
        print(f"  크기 범위: {min(sizes)}~{max(sizes)}자")


📊 레벨별 문서 분석 결과:

🔹 MAJOR 레벨:
  총 문서: 46개
  PNS 관련: 1개 (2.2%)
  평균 크기: 432자
  크기 범위: 2~4121자

🔹 MEDIUM 레벨:
  총 문서: 88개
  PNS 관련: 5개 (5.7%)
  평균 크기: 323자
  크기 범위: 2~1448자

🔹 MINOR 레벨:
  총 문서: 824개
  PNS 관련: 32개 (3.9%)
  평균 크기: 464자
  크기 범위: 2~5274자


## 6. PNS + purchaseState 연결 문서 분석


In [110]:
# PNS + purchaseState 동시 포함 문서 찾기
pns_purchase_docs = []
for doc in documents:
    is_pns = doc.metadata.get('contains_pns', False)
    has_purchase = 'purchasestate' in doc.page_content.lower()
    
    if is_pns and has_purchase:
        pns_purchase_docs.append(doc)

print(f"🎯 PNS + purchaseState 동시 포함 문서: {len(pns_purchase_docs)}개")
print("=" * 60)

# 상위 5개 문서 상세 분석
for i, doc in enumerate(pns_purchase_docs[:5]):
    hierarchy = doc.metadata.get('title_hierarchy', 'Unknown')
    level = doc.metadata.get('hierarchy_level', 'unknown')
    chunk_size = doc.metadata.get('chunk_size', 0)
    
    print(f"\n📄 PNS+purchaseState 문서 #{i+1}")
    print(f"  레벨: {level}")
    print(f"  계층: {hierarchy}")
    print(f"  크기: {chunk_size}자")
    
    # purchaseState 주변 컨텍스트 추출
    content_lower = doc.page_content.lower()
    purchase_idx = content_lower.find('purchasestate')
    if purchase_idx != -1:
        start = max(0, purchase_idx - 100)
        end = min(len(doc.page_content), purchase_idx + 150)
        context = doc.page_content[start:end].replace('\n', ' ')
        print(f"  컨텍스트: ...{context}...")

if len(pns_purchase_docs) >= 3:
    print(f"\n✅ 성공: PNS와 purchaseState 연결 문제 해결!")
    print(f"   MultiLevelSplittingStrategy가 효과적으로 작동했습니다.")
else:
    print(f"\n⚠️  개선 필요: PNS-purchaseState 연결 문서가 부족합니다.")


🎯 PNS + purchaseState 동시 포함 문서: 3개

📄 PNS+purchaseState 문서 #1
  레벨: minor
  계층: 07. PNS(Payment Notification Service) 이용하기 > **PNS 상세** > PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)
  크기: 2202자
  컨텍스트: ...시간(ms)                                                  |                                       | | purchaseState      | String        | COMPLETED : 결제완료 / CANCELED : 취소                                              |                                  ...

📄 PNS+purchaseState 문서 #2
  레벨: minor
  계층: 07. PNS(Payment Notification Service) 이용하기 > **PNS 상세** > PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)
  크기: 1007자
  컨텍스트: ...seId":"SANDBOX3000000004564", "developerPayload":"OS_000211234", "purchaseTimeMillis":24431212233, "purchaseState":"COMPLETED", "price":"10000", "priceCurrencyCode":"KRW" "productName":"GOLD100(+20)" "paymentTypeList":[ { "paymentMethod":"DCB", "amou...

📄 PNS+purchaseState 문서 #3
  레벨: minor
  계층: 07. PNS(Payment Notification Service) 이용하기 > **PNS 상세** > 

## 7. 간단한 검색 테스트


In [ ]:
# 간단한 키워드 기반 검색 함수
def simple_search(documents, query, max_results=5):
    """간단한 키워드 기반 검색"""
    results = []
    query_lower = query.lower()
    
    for doc in documents:
        score = 0
        content_lower = doc.page_content.lower()
        hierarchy = doc.metadata.get('title_hierarchy', '').lower()
        
        # 기본 키워드 매칭
        if 'pns' in query_lower and 'pns' in content_lower:
            score += 10
        if 'purchasestate' in query_lower and 'purchasestate' in content_lower:
            score += 10
        
        # 계층 제목 매칭 보너스
        if 'pns' in query_lower and 'pns' in hierarchy:
            score += 15
        if 'purchasestate' in query_lower and 'purchasestate' in hierarchy:
            score += 15
        
        # PNS 관련 문서 보너스
        if doc.metadata.get('contains_pns', False):
            score += 5
        
        # 레벨별 가중치
        level = doc.metadata.get('hierarchy_level', 'minor')
        if level == 'major':
            score *= 1.2
        elif level == 'medium':
            score *= 1.1
        
        if score > 0:
            results.append((score, doc))
    
    # 점수순 정렬
    results.sort(key=lambda x: x[0], reverse=True)
    return [doc for score, doc in results[:max_results]]

# 핵심 쿼리 테스트
test_query = "PNS 메시지 서버 규격의 purchaseState는 어떤 값으로 구성되나요?"
print(f"🔍 테스트 쿼리: {test_query}")
print("=" * 80)

search_results = simple_search(documents, test_query, max_results=5)

print(f"\n📊 검색 결과: {len(search_results)}개")
pns_count = sum(1 for doc in search_results if doc.metadata.get('contains_pns', False))
purchase_count = sum(1 for doc in search_results if 'purchasestate' in doc.page_content.lower())
both_count = sum(1 for doc in search_results 
                if doc.metadata.get('contains_pns', False) and 'purchasestate' in doc.page_content.lower())

print(f"  PNS 관련: {pns_count}/{len(search_results)}")
print(f"  purchaseState 포함: {purchase_count}/{len(search_results)}")
print(f"  두 조건 모두: {both_count}/{len(search_results)}")

# 검색 결과 상세 보기
for i, doc in enumerate(search_results[:3]):
    hierarchy = doc.metadata.get('title_hierarchy', 'Unknown')
    level = doc.metadata.get('hierarchy_level', 'unknown')
    is_pns = doc.metadata.get('contains_pns', False)
    has_purchase = 'purchasestate' in doc.page_content.lower()
    
    print(f"\n📄 결과 #{i+1} ({level})")
    print(f"  계층: {hierarchy}")
    print(f"  PNS: {'✅' if is_pns else '❌'} | purchaseState: {'✅' if has_purchase else '❌'}")
    print(f"  내용: {doc.page_content[:200].replace(chr(10), ' ')}...")


## 8. 성능 요약 및 결론


In [ ]:
print("🏆 MultiLevelSplittingStrategy 성능 요약")
print("=" * 80)

# 전체 통계
total_docs = len(documents)
total_pns = len([doc for doc in documents if doc.metadata.get('contains_pns', False)])
total_purchase = len([doc for doc in documents if 'purchasestate' in doc.page_content.lower()])
total_both = len(pns_purchase_docs)

print(f"\n📊 전체 문서 통계:")
print(f"  총 문서 수: {total_docs:,}개")
print(f"  PNS 관련 문서: {total_pns}개 ({total_pns/total_docs*100:.1f}%)")
print(f"  purchaseState 포함: {total_purchase}개 ({total_purchase/total_docs*100:.1f}%)")
print(f"  PNS + purchaseState: {total_both}개 ({total_both/total_docs*100:.1f}%)")

# 레벨별 분포
print(f"\n📈 레벨별 분포:")
for level in ['major', 'medium', 'minor']:
    level_docs = [doc for doc in documents if doc.metadata.get('hierarchy_level') == level]
    level_pns = [doc for doc in level_docs if doc.metadata.get('contains_pns', False)]
    level_avg_size = sum(doc.metadata.get('chunk_size', 0) for doc in level_docs) / len(level_docs) if level_docs else 0
    
    print(f"  {level.upper()}: {len(level_docs)}개 (PNS: {len(level_pns)}개, 평균: {level_avg_size:.0f}자)")

# 핵심 성과 지표
print(f"\n🎯 핵심 성과 지표:")
if total_purchase > 0:
    connection_rate = total_both/total_purchase*100
    print(f"  PNS-purchaseState 연결 성공률: {connection_rate:.1f}%")
else:
    print(f"  PNS-purchaseState 연결 성공률: 0%")

print(f"  연결된 문서 수: {total_both}개")

# 성과 평가
if total_both >= 5:
    print(f"\n✅ 우수: PNS 대제목과 하위 섹션 연결 문제 완전 해결!")
    print(f"   MultiLevelSplittingStrategy가 매우 효과적으로 작동했습니다.")
elif total_both >= 2:
    print(f"\n⚡ 양호: PNS-purchaseState 연결 일부 성공")
    print(f"   추가 최적화를 통해 더 나은 결과를 얻을 수 있습니다.")
else:
    print(f"\n⚠️ 개선 필요: 연결 문제가 지속됩니다.")

print(f"\n💡 MultiLevelSplittingStrategy 핵심 특징:")
print(f"  1. 계층별 차등 청크 크기: Major(2000자) > Medium(1200자) > Minor(800자)")
print(f"  2. PNS 컨텍스트 명시적 포함: '[PNS 관련]: 이 내용은 PNS...'")
print(f"  3. 제목 계층 구조 보존: '[계층]: PNS > 메시지 규격 > ...'")
print(f"  4. 계층적 검색 점수 계산: 제목 매칭 시 보너스 점수")

print(f"\n🚀 적용 권장사항:")
print(f"  ✅ MultiLevelSplittingStrategy를 메인 RAG 파이프라인에 즉시 적용")
print(f"  ✅ 계층적 점수 계산을 활용한 검색 시스템 구축") 
print(f"  ✅ PNS 관련 쿼리에 특화된 검색 로직 구현")
